## SetUp

In [2]:
# Librerías para manejo de datos
import numpy as np
import pandas as pd

# Librerías para preprocesamiento y modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# Librerías para métricas y evaluación
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Librerías para visualización
import matplotlib.pyplot as plt

# Librerías para redes neuronales y optimización
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Librerías para ajuste de hiperparámetros
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt

# Librerías estadísticas
from scipy.stats import pointbiserialr


In [3]:
#importacion df_final
df=pd.read_csv('../data/processed/v3_final_merge.csv')

## Data processing and exploration

In [4]:
df

,edad,sexo,nivel_educativo,fumador,actividad_fisica,vive,diabetes,hipertension,obesidad,cancer,...,region_Sur,tipo_empleo_Desempleado,tipo_empleo_Privado,tipo_empleo_Público,tipo_empleo_Temporal,continente_America,continente_Antarctica,continente_Asia,continente_Europe,continente_Oceania
0,69,0,1,1,0,1,1,1,1,0,...,0,0,0,1,0,0,0,1,0,0
1,32,1,2,0,1,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0
2,89,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,78,1,1,1,0,0,1,0,1,1,...,0,0,0,0,1,0,0,1,0,0
4,38,0,3,1,1,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,21,1,2,0,2,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
49996,35,0,3,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
49997,46,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
49998,56,1,3,1,1,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [5]:
df.shape

(50000, 48)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   edad                     50000 non-null  int64  
 1   sexo                     50000 non-null  int64  
 2   nivel_educativo          50000 non-null  int64  
 3   fumador                  50000 non-null  int64  
 4   actividad_fisica         50000 non-null  int64  
 5   vive                     50000 non-null  int64  
 6   diabetes                 50000 non-null  int64  
 7   hipertension             50000 non-null  int64  
 8   obesidad                 50000 non-null  int64  
 9   cancer                   50000 non-null  int64  
 10  enfermedad_cardiaca      50000 non-null  int64  
 11  epoc                     50000 non-null  int64  
 12  glucosa                  50000 non-null  float64
 13  colesterol               50000 non-null  float64
 14  trigliceridos         

## Correlation

A continuación, vamos a estudiar la **correlación** de las variables con la clase objetivo que queremos predecir, es decir, si el paciente **vive o no**. Este análisis nos permitirá identificar qué variables tienen una relación más fuerte con la variable objetivo, lo que puede ser útil para seleccionar las características más relevantes para nuestra red neuronal.

In [7]:
X = df.drop(columns=['vive'])
y = df['vive']

# Calcular correlaciones
correlaciones = {}

for col in X.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        try:
            corr, _ = pointbiserialr(df[col], y)
            correlaciones[col] = corr
        except:
            correlaciones[col] = None
    else:
        correlaciones[col] = None

# Convertir a DataFrame
corr_df = pd.DataFrame.from_dict(correlaciones, orient='index', columns=['Correlación con vive'])

# Ordenar por valor absoluto
corr_df = corr_df.reindex(corr_df['Correlación con vive'].abs().sort_values(ascending=False).index)

corr_df

,Correlación con vive
cancer,-0.272906
gastos_salud,-0.262333
enfermedad_cardiaca,-0.242263
creatinina,-0.087198
epoc,-0.062929
hipertension,-0.060034
diabetes,-0.056181
leucocitos,-0.052074
fumador,-0.042654
glucosa,-0.035429


**Selección de Variables Significativas**

Para evitar introducir ruido en el modelo con datos poco significativos, inicialmente trabajaremos con las **15 variables más correlacionadas** con la variable objetivo (*vive*). Este enfoque nos ayudará a construir un modelo más eficiente y con mejor capacidad de generalización.


In [8]:
# Obtener los nombres de las 15 variables más correlacionadas
top_15_vars = corr_df.head(15).index.tolist()

# Crear nuevo DataFrame con esas variables y 'vive'
df_top15 = df[top_15_vars + ['vive']]

df_top15.head()

,cancer,gastos_salud,enfermedad_cardiaca,creatinina,epoc,hipertension,diabetes,leucocitos,fumador,glucosa,colesterol,edad,actividad_fisica,obesidad,ingresos_mensuales,vive
0,0,450,1,0.97,0,1,1,6.12,1,108.81,226.16,69,0,1,2699.72,1
1,0,150,0,0.85,0,0,0,6.97,0,74.23,148.89,32,1,0,3403.55,0
2,0,150,0,1.59,1,0,0,7.54,1,88.61,177.53,89,0,0,2422.41,1
3,1,450,1,1.26,0,0,1,8.56,1,90.92,203.97,78,0,1,1695.42,0
4,0,150,0,0.88,0,0,0,8.20,1,69.17,154.33,38,1,0,1709.51,1


In [9]:
df_top15.columns

Index(['cancer', 'gastos_salud', 'enfermedad_cardiaca', 'creatinina', 'epoc',
       'hipertension', 'diabetes', 'leucocitos', 'fumador', 'glucosa',
       'colesterol', 'edad', 'actividad_fisica', 'obesidad',
       'ingresos_mensuales', 'vive'],
      dtype='object')

## Imbalance

Antes de proceder con el modelado, es importante analizar si la clase objetivo (*vive*) está desbalanceada. Un **desbalance en las clases puede afectar negativamente el rendimiento del modelo**, especialmente en términos de su capacidad para predecir correctamente la clase minoritaria. 


In [10]:
neg, pos = np.bincount(df['vive'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n    Negative: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total, neg, 100 * neg / total))

Examples:
    Total: 50000
    Positive: 43584 (87.17% of total)
    Negative: 6416 (12.83% of total)



Como podemos ver tenemos muy pocas muestras de aquellos pacientes que fallecen por lo que va a ser complicado estimar esta clase(0)